# MRE: Federal CTC Can Reduce MA State Tax (Edge Case)

This notebook demonstrates how federal CTC expansion *could* reduce Massachusetts state tax if PolicyEngine's itemization logic used `<=` instead of `<`.

In [ ]:
from policyengine_us import Simulation
from policyengine_us.model_api import *
import numpy as np

## The Issue

PolicyEngine's current itemization logic:
```python
return tax_liability_if_itemizing < tax_liability_if_not_itemizing
```

This means when taxes are exactly equal, it doesn't itemize. However, in the real data (household 4428), we see itemization occurring when taxes are equal or very close.

In [ ]:
# Create a minimal household
situation = {
    "people": {
        "parent": {
            "age": {2026: 45},
            "employment_income": {2026: 17_237},
            "medical_out_of_pocket_expenses": {2026: 3_339},
        },
        "spouse": {
            "age": {2026: 43},
        },
        "child1": {"age": {2026: 13}},
        "child2": {"age": {2026: 12}},
        "child3": {"age": {2026: 6}},
        "child4": {"age": {2026: 4}},
    },
    "tax_units": {
        "tax_unit": {
            "members": ["parent", "spouse", "child1", "child2", "child3", "child4"],
        }
    },
    "households": {
        "household": {
            "members": ["parent", "spouse", "child1", "child2", "child3", "child4"],
            "state_code": {2026: "MA"},
        }
    },
}

## Demonstrate Current Behavior

In [ ]:
# Senate Finance reforms with CTC
reform = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48_900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2_200},
}, country_id="us")

sim = Simulation(situation=situation, reform=reform)

# Check federal tax
fed_item = sim.calculate('tax_liability_if_itemizing', 2026)[0]
fed_std = sim.calculate('tax_liability_if_not_itemizing', 2026)[0]
itemizes = bool(sim.calculate('tax_unit_itemizes', 2026)[0])

print("CURRENT BEHAVIOR:")
print(f"  Federal tax if itemizing: ${fed_item:,.2f}")
print(f"  Federal tax if standard: ${fed_std:,.2f}")
print(f"  Difference: ${fed_item - fed_std:,.2f}")
print(f"  Itemizes: {itemizes}")
print(f"  MA state tax: ${sim.calculate('ma_income_tax', 2026)[0]:,.2f}")

## What Would Happen with <= Logic

In [ ]:
# If PolicyEngine itemized when federal taxes are equal
if abs(fed_item - fed_std) < 0.01:
    print("\nIF POLICYENGINE USED <= INSTEAD OF <:")
    print("  Would itemize: True")
    
    # Calculate what MA tax would be with itemization
    # MA allows medical expense deduction for itemizers
    med_expense = sim.calculate('medical_expense_deduction', 2026)[0]
    print(f"  Medical expense deduction: ${med_expense:,.2f}")
    
    # MA Part B exemption includes medical expenses for itemizers
    # Base exemption + medical expense deduction
    base_exemption = 12_800  # for married filing jointly
    ma_exemption_with_medical = base_exemption + med_expense
    
    print(f"  MA Part B exemption would be: ${ma_exemption_with_medical:,.2f}")
    print(f"  Additional exemption: ${med_expense:,.2f}")
    print(f"  MA tax savings (5% rate): ${med_expense * 0.05:,.2f}")
    
    print("\nThis matches the $56.41 reduction seen in household 4428!")

## Proposed Fix

The issue could be resolved by changing the itemization logic to:
```python
return tax_liability_if_itemizing <= tax_liability_if_not_itemizing
```

This would:
1. Allow itemization when federal taxes are equal
2. Enable MA medical expense deduction for these edge cases
3. Match the behavior seen in the actual data